In [2]:
import torch
from d2l import torch as d2l

## 1 多输入通道

In [5]:
def corr2d(X, K):  #@save K for kernel，X for local image 
    """计算二维互相关运算。"""
    h, w = K.shape 
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

def corr2d_multi_in(X, K): # X 为n通道图像，K为对应n通道内核, 返回的是单通道的图像
    return sum(corr2d(x, k) for x, k in zip(X, K))

In [11]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
print(corr2d_multi_in(X, K))

tensor([[ 56.,  72.],
        [104., 120.]])


## 2 多输出通道

In [8]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [12]:
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
K = torch.stack((K, K + 1, K + 2), 0) # 构造3输出通道，stack创建新维度
print(K.shape)
print(corr2d_multi_in_out(X, K))

torch.Size([3, 2, 2, 2])
tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])


## 3 1×1卷积层

In [30]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w)) # h*w相当于展平，用于全连接
    K = K.reshape((c_o, c_i)) # 全连接层的参数
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [31]:
X = torch.normal(0, 1, (3, 3, 3)) # 3通道3*3图像，按照标准正态分布
K = torch.normal(0, 1, (2, 3, 1, 1)) # 正态分布的（01）的2out，3in 的1*1卷积核

In [35]:
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6